# AI CALCULATOR

🎲🎲🎲

Complete a notebook to classify numbers (1-10) and 3 basic mathematics operators (+,-, and x)

## IMPORT LIBRARIES

In [ ]:
import tensorflow as tf
print(tf.__version__) #If not Tensorflow 2 !pip install tensorflow-gpu==2.1.0

import shutil # copy, move file
import os
import pathlib

import random
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.pylab as plt
%matplotlib inline

2.3.0


In [ ]:
# Path to data folder
DATA_FOLDER = ""

## LOAD DATA

In [ ]:
# Path to dataset
data_root = pathlib.Path(DATA_FOLDER)

# Find all image's path
all_image_paths = list(data_root.glob("**/*"))

# Check if the path contains file
all_image_paths = [str(path) for path in all_image_paths if path.is_file()]

# Make sure data is well-shuffled
random.shuffle(all_image_paths)

# Count the number of images
image_count = len(all_image_paths)

print(f'There is total {image_count} images.')

Our folder may contain other files that not image. Cross them out. 

In [ ]:
# Read image files and ignore DS_Store
for index, image_path in enumerate(all_image_paths):
    if (image_path.split('.')[1] != "DS_Store") :
        try:
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image, channels=3)
        except:
            print(f'Image {all_image_paths[index]} is not readable.')
        
        if index % 3000 == 0 or index == image_count-1: 
            print(f'Finished scanning {index+1}/{image_count} files.')
    else:
        all_image_paths.remove(image_path)

Now your all the image paths are in all_image_paths and ready for the training!

## PREPROCESSING 

### Labels

**YOUR TASK:** CREATE A LIST all_image_labels that contains all the labels corresponding to the paths in all_image_paths. 

Hint: The label is the image's folder name - `pathlib.Path(path).parent.name`

In [ ]:
# YOUR CODE HERE

**YOUR TASK**: Count the number of images in each class.

In [ ]:
# YOUR CODE HERE 

### Images

**YOUR TASK**: Define the function that uses Tensorflow to 
* Read and decode image
* Convert to grayscale
* Normalize
* Invert image (white background to black background)

In [ ]:
# YOUR CODE HERE

**YOUR TASK**: Plot a random image to see if the read and preprocess code works.

In [ ]:
# YOUR CODE HERE

## CREATE DATASET

### Split dataset

**YOUR TASK**: Split the dataset into train-test with your chosen ratio. 

In [ ]:
# YOUR CODE HERE

### Flow dataset using tf.data.Dataset
**YOUR TASK**: Complete the code

In [ ]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

def create_ds(all_image_paths, all_image_labels):
    
    # prepare the dataset for pipeline with tensor_slices
    ds = ______
    
    # map the load_and_preprocess_from_path_label function to the pipeline
    image_label_ds = ds._________

    # Setting a shuffle buffer size as large as the dataset ensures that the data is 
    # well-shuffled, so the order of training will be different for every epoch.
    ds = image_label_ds.__________
    
    # `prefetch` lets the dataset fetch batches in the background while the model is training.
    ds = ds.________
    
    return ds

In [ ]:
# create dataset for train set and test set
train_ds = create_ds(train_image_paths, train_image_labels)
test_ds = create_ds(test_image_paths, test_image_labels)

## MODEL

**YOUR TASK**: Build a CNN model and compile it.

In [ ]:
# YOUR CODE HERE

## TRAIN

### Callbacks

In [ ]:
# Batch Stats Collector that inherits Keras Callback
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_acc = []
        self.batch_val_losses = []
        self.batch_val_acc = []

    def on_train_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['accuracy'])
        # reset_metrics: the metrics returned will be only for this batch. 
        # If False, the metrics will be statefully accumulated across batches.
        self.model.reset_metrics()
  
    def on_test_batch_end(self, batch, logs=None):
        self.batch_val_losses.append(logs['loss'])
        self.batch_val_acc.append(logs['accuracy'])
        # reset_metrics: the metrics returned will be only for this batch. 
        # If False, the metrics will be statefully accumulated across batches.
        self.model.reset_metrics()

# Plot learning curve 
def plot_stats(training_stats, val_stats, x_label='Training Steps', stats='loss'):
    stats, x_label = stats.title(), x_label.title()
    legend_loc = 'upper right' if stats=='loss' else 'lower right'
    training_steps = len(training_stats)
    test_steps = len(val_stats)

    plt.figure()
    plt.ylabel(stats)
    plt.xlabel(x_label)
    plt.plot(training_stats, label='Training ' + stats)
    plt.plot(np.linspace(0, training_steps, test_steps), val_stats, label='Validation ' + stats)
    plt.ylim([0,max(plt.ylim())])
    plt.legend(loc=legend_loc)
    plt.show()

### Train Session

In [ ]:
# Calculate the number of steps to finish one epoch
steps_per_epoch = ________
val_steps_per_epoch = __________

# Define callback
batch_stats_callback = CollectBatchStats()

In [ ]:
history = cnn_model.fit(train_ds, 
                        epochs=_____, 
                        steps_per_epoch=steps_per_epoch,
                        callbacks = [batch_stats_callback],
                        validation_data=test_ds,
                        validation_steps=val_steps_per_epoch)

In [ ]:
# Visualize training session on batch 
plt.figure(figsize = (15, 10))

plot_stats(batch_stats_callback.batch_losses, batch_stats_callback.batch_val_losses, stats='loss')
plot_stats(batch_stats_callback.batch_acc, batch_stats_callback.batch_val_acc, stats='accuracy')

In [ ]:
plt.figure(figsize = (15, 10))

plot_stats(history.history['loss'], history.history['val_loss'], x_label='Epochs', stats='loss')
plot_stats(history.history['accuracy'], history.history['val_accuracy'], x_label='Epochs', stats='accuracy');

## TEST

In [ ]:
# Take one batch to predict
for image_batch, label_batch in test_ds.take(1):
    print("Image batch shape: ", image_batch.shape)
    print("Label batch shape: ", label_batch.shape)

**YOUR TASK**: Predict image_batch

In [ ]:
# YOUR CODE HERE

## EXPORT

**YOUR TASK**: Export the model

In [ ]:
# YOUR CODE HERE